# Network Creation

In [1]:
#!pip install torch ./Gym-Wordle

In [2]:
import torch
import matplotlib.pyplot as plt
from torch import normal

In [3]:
from torch import nn

class ActorNetwork(nn.Module):
    """LSTM RNN for generating words for wordle solver"""
    def __init__(self, input_size, output_size=2):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, output_size),
        )
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        y = self.network(x)
        return y

class CriticNet(nn.Module):
    """Network representing the critic"""
    def __init__(self, in_shape):
        super().__init__()
        self.v_network = nn.Sequential(
            nn.Linear(in_shape, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
            )

    def forward(self, x):
        return self.v_network(x)

# Figuring out the environment

In [4]:
import gym
import gym_wordle.utils
import numpy as np
wordle = gym.make('Wordle-v0')

# Competing A2C solvers

In [5]:
def train(replay, q_val):
    vals = torch.vstack(replay.vals)
    
    rewards_tensor = torch.tensor(np.asarray(replay.rewards, dtype=np.float32))
    is_terminal_tensor = torch.tensor(np.asarray(replay.dones, dtype=np.int32))
        
    q_vals_tensor = rewards_tensor + discount_factor * q_val * (1 - is_terminal_tensor)
        
    advantage = q_vals_tensor - vals
    
    critic_loss = (advantage ** 2).mean()
    # critic_loss.requires_grad = True
    adam_critic.zero_grad()
    critic_loss.backward(retain_graph=True)
    adam_critic.step()
    
    
    log_probabs_tensor = torch.vstack(replay.log_probs)
    actor_loss = (-log_probabs_tensor*advantage.detach()).mean()
    # actor_loss.requires_grad = True
    adam_actor.zero_grad()
    actor_loss.backward()
    adam_actor.step()

In [6]:
class Advantage_ActorCritic():
    def __init__(self, world: gym.Env, policy_net: ActorNetwork, critic,
                 encoder, policy_alpha,  gamma, max_reward):
        # environment info
        self.world = world
        self.encoder = encoder
        self.max_reward = max_reward

        # actor and critic
        self.actor = policy_net
        self.critic = critic
        self.error_buffer = list()
        self.policy_optimizer = torch.optim.Adam(policy_net.network.parameters(), lr=policy_alpha)

        # training info
        self.gamma = gamma
        self.episodes = 0
        self.step = -1
        self.guesses = np.zeros(world.action_space.n)

    def train(self, iterations):
        converged = False
        rewards = list()
        recents = torch.zeros(10)
        i = 0
        while not converged:
            r = self.episode()
            rewards.append(r)

            if i < 10:
                recents[i] = r
                i += 1

            else:
                recents.roll(-1, 0)
                recents[9] = r
            
            # convergence check
            if len(rewards) > 10 and recents.mean() >= self.max_reward:
                converged = True
            converged = True if iterations == self.episodes else False
        
        plt.plot(rewards)
        plt.show()

    def episode(self, training=True):
        done = False
        state = self.world.reset().copy()
        episode_reward = 0
        self.episodes += 1
        self.step = -1
        while not done:
            self.step += 1
            # take on policy action
            encoded = self.encoder(state)
            dist = self.actor(encoded)

            action = normal(dist[0], dist[1])
            action = int(action.clip(0, self.world.action_space.n-1).round().item())
            self.guesses[action] += 1
            state_prime, reward, done, _ = self.world.step(action)

            # fill buffer
            self.error_buffer.append((state, state_prime, reward, dist))

            # prepare for next iteration
            episode_reward += reward
            state = state_prime.copy()
            if training:
                self.__net_update()

        return episode_reward

    # Actor error
    def policy_error(self, prob, error):
        log_probabs_tensor = prob.reshape(-1)
        return (-log_probabs_tensor * error).mean()

    def __net_update(self):
        # calculate error for policy and critic
        self.actor.network.zero_grad()

        state, state_prime, reward, prob = self.error_buffer.pop(0)
        loss = self.critic(state_prime[self.step])
        policy_error = self.policy_error(prob, loss) 

        # backprop
        policy_error.backward()
        self.policy_optimizer.step()

# Use entropy minimization to develop a critic

In [7]:
entropies = list()
def entropy_critic(guess: np.ndarray):
	remaining_space = gym_wordle.utils.get_words("guess")
	# get guess and the values
	split = np.split(guess, 2)
	g = split[0]
	vals = split[1]
	print(g, vals)

	# get complete matches (i.e. matches based on position)
	matches = np.argwhere(vals==1).flatten()
	print(matches)
	for i in matches:
		#print(np.nonzero(remaining_space[:, i] == g[i]))
		remaining_space = remaining_space[np.nonzero(remaining_space[:, i] == g[i])] 
	print(remaining_space.shape)
	# find remaining words that this letter
	exists = np.argwhere(vals==2).flatten()
	print(exists)
	for i in exists:
		print("letter", g[i])
		print(np.any(remaining_space, 1, where=g[i]).shape , " exist")
		remaining_space = remaining_space[np.any(remaining_space, 1, where=g[i])]
	print(remaining_space.shape)

	# we know what letters we DON'T have
	dne = np.argwhere(vals==3).flatten()
	print(dne)
	for i in dne:
		print(np.any(remaining_space, 1, where=g[i]).shape , "dne")  # this is returning entire dictionary
		remaining_space = remaining_space[np.logical_not(np.any(remaining_space, 1, where=g[i]))]

	# do calc
	p = float(remaining_space.shape[0]) / float(wordle.action_space.n)
	print(p)
	entropy = -p * np.log(p)
	entropies.append(entropy)
	return entropy

In [13]:
#wordle.reset()
#obs = wordle.step(0)
wordle.render()
entropy_critic(obs[0][0])

AAHED
AAHED
AAHED
     
     
     
[1 1 8 5 4] [3 3 2 2 3]
[]
(12972, 5)
[2 3]
letter 8
(12972,)  exist
letter 5
(12972,)  exist
(12972, 5)
[0 1 4]
(12972,) dne
(0,) dne
(0,) dne
0.0
AAHED
AAHED
AAHED
     
     
     


/var/folders/pz/mmdg2qc54x9gxppy369prgy80000gn/T/ipykernel_97099/4151420395.py:37: RuntimeWarning: divide by zero encountered in log
  entropy = -p * np.log(p)
/var/folders/pz/mmdg2qc54x9gxppy369prgy80000gn/T/ipykernel_97099/4151420395.py:37: RuntimeWarning: invalid value encountered in double_scalars
  entropy = -p * np.log(p)


# Solving the Environment

In [9]:
critic = entropy_critic
actor = ActorNetwork(60, 2)
agent = Advantage_ActorCritic(wordle, actor, critic, 
lambda x: torch.tensor(x.flatten()).to(torch.float32), 1e-3, 0.9, -4)

In [10]:
agent.episode()

[1 1 8 5 4] [3 3 2 2 3]
[]
(12972, 5)
[2 3]
letter 8
(12972,)  exist
letter 5
(12972,)  exist
(12972, 5)
[0 1 4]
(12972,) dne
(0,) dne
(0,) dne
0.0


/var/folders/pz/mmdg2qc54x9gxppy369prgy80000gn/T/ipykernel_97099/4151420395.py:37: RuntimeWarning: divide by zero encountered in log
  entropy = -p * np.log(p)
/var/folders/pz/mmdg2qc54x9gxppy369prgy80000gn/T/ipykernel_97099/4151420395.py:37: RuntimeWarning: invalid value encountered in double_scalars
  entropy = -p * np.log(p)


ValueError: cannot convert float NaN to integer

In [ ]:
agent.train(2000)
plt.plot(entropies)
plt.show()